# imports

In [18]:
import glob, os, sys, io, skvideo.io, argparse, math, datetime, ffmpy, shutil, wikipedia
from google.cloud import videointelligence_v1 as videointelligence
from google.cloud import vision
from google.cloud import storage
from google.cloud.vision import types
from PIL import Image, ImageDraw
import numpy as np

# feature extraction with google cloud video intelligence api

In [ ]:
def analyze_labels(movie_to_process, bucket_name):
    
    #path to video input file and JSON output file
    input_uri = 'gs://' + bucket_name + '/video_processing/' + movie_to_process
    
    
    video_client = videointelligence.VideoIntelligenceServiceClient()
    
    # request extracted features
    features = [videointelligence.enums.Feature.LABEL_DETECTION, videointelligence.enums.Feature.SPEECH_TRANSCRIPTION]

    # request shot-level and frame_level labels, assign to label detection config
    label_mode = videointelligence.enums.LabelDetectionMode.SHOT_AND_FRAME_MODE
    label_config = videointelligence.types.LabelDetectionConfig(label_detection_mode=label_mode)
    
    # set language context to British english FOR SHERLOCK
    speech_config = videointelligence.types.SpeechTranscriptionConfig(language_code='en-GB')
    
    # pass label detection and speech transcription configs to context 
    context = videointelligence.types.VideoContext(label_detection_config=label_config, SpeechTranscriptionConfig=speech_config)

    # define analysis from parameters
    operation = video_client.annotate_video(input_uri, features=features, video_context=context)
    print('\nprocessing video')
    
    # check if operation is done every 90s, return if done
    result = operation.result(timeout=90)
    print('\nfinished processing video')
    
    return result


In [ ]:
# movie_to_process is sherlock_video.mp4
# bucket is sherlock_movie

# where do I want to output

In [ ]:

    # time offset from beginning of video where label is located
    frame_offsets = []
 







    # Process frame level label annotations
    frame_labels = result.annotation_results[0].frame_label_annotations
    for i, frame_label in enumerate(frame_labels):
       for category_entity in frame_label.category_entities:
            # look for categories that contain person regardless of situation
            if (category_entity.description == 'person'):
                # Each frame_label_annotation has many frames,
                # but we keep information only about the first one
                frame = frame_label.frames[0]
                time_offset = (frame.time_offset.seconds +
                               frame.time_offset.nanos / 1e9)
                print('\tFirst frame time offset: {}s'.format(time_offset))
                print('\tFirst frame confidence: {}'.format(frame.confidence))
                print('\n')
                frame_offsets.append(time_offset)
                
    return(sorted(set(frame_offsets)))

## scene details

In [ ]:
# Google Cloud Video Intelligence returning 

## characters on screen (analogous to name - all)

## camera angle

## indoor vs outdoor

## character speaking

## location

## music presence

## name - focus

## words on screen